In [1]:
from retrievers import VectorStoreRetriever, CustomContextualCompressionRetriever, CustomMultiQueryRetriever
from composers import ChainOfThoughtComposer
from config import openai_key
from functions import devrev_functions
from examples import example_queries

from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.llms import OpenAI, HuggingFaceTextGenInterface
from langchain.chat_models import ChatOpenAI

ImportError: cannot import name 'HuggingFaceTextGenInterface' from 'langchain.llms' (c:\Users\kayde\OneDrive\Desktop\devrev-openai-classic\.venv\lib\site-packages\langchain\llms\__init__.py)

In [ ]:
openai_embeddings = OpenAIEmbeddings(
    openai_api_key = openai_key
)

openai_chat = ChatOpenAI(
    openai_api_key = openai_key,
    temperature=0.0,
    max_tokens=1000
)

retriever = VectorStoreRetriever(
    embeddings=openai_embeddings,
    name = "vs_rtr_01",
    init_functions=devrev_functions.copy()
)


examples = "".join(
    [f"Query: {query}\nOutput: {output}\n" for query, output in example_queries[:3]]
)

composer = ChainOfThoughtComposer(
    retriever=retriever,
    name = "cotc_01",
    init_functions=devrev_functions.copy()
)
